In [37]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

from sklearn.model_selection import train_test_split
from ast import literal_eval

import matplotlib.pyplot as plt

In [3]:
transactions_train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')

In [5]:
transactions_train['t_dat_datetime']  = pd.to_datetime(transactions_train['t_dat'])

transactions_train['t_month'] = transactions_train['t_dat_datetime'].dt.month

transactions_train['t_year']  = transactions_train['t_dat_datetime'].dt.year

In [6]:
transactions_train['customer_max_tran']  = transactions_train.\
groupby(['customer_id'])['t_dat_datetime'].transform('max')

In [97]:
cust_data = transactions_train[(transactions_train['customer_max_tran'].dt.year==2020)&
                                     (transactions_train['customer_max_tran'].dt.month == 9)]

cust_data['article_id'] = 'a'+cust_data['article_id'].astype('str')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [87]:
cust_data_2020_9 = cust_data.groupby(['customer_id','article_id']).agg(tran_count = ('t_dat_datetime','nunique')).\
assign(Rank = lambda x : x.groupby('customer_id')['tran_count'].rank(method='first', ascending=False)
      ).\
reset_index().\
query('Rank <= 12').\
groupby('customer_id').agg(prods = ('article_id','unique')).reset_index()

In [95]:
article_df =  pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
customer_df = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')

article_df['article_id'] = 'a'+article_df['article_id'].astype('str')

In [96]:
article_df.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,a108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,a108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,a108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,a110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,a110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [98]:
cust_data = pd.merge(pd.merge(cust_data ,
         article_df[['article_id','product_group_name']],
         left_on = 'article_id',
         right_on = 'article_id'
        ),
         customer_df[['customer_id','age']],
         left_on = 'customer_id',
         right_on = 'customer_id'
        )

In [99]:
cust_data.head()

,t_dat,customer_id,article_id,price,sales_channel_id,t_dat_datetime,t_month,t_year,customer_max_tran,product_group_name,age
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,a663713001,0.050831,2,2018-09-20,9,2018,2020-09-15,Underwear,24.0
1,2018-09-24,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,a663713001,0.050831,2,2018-09-24,9,2018,2020-09-15,Underwear,24.0
2,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,a541518023,0.030492,2,2018-09-20,9,2018,2020-09-15,Underwear,24.0
3,2020-02-03,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,a351484002,0.022017,2,2020-02-03,2,2020,2020-09-15,Swimwear,24.0
4,2020-02-03,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,a351484002,0.022017,2,2020-02-03,2,2020,2020-09-15,Swimwear,24.0


In [100]:
cust_features = cust_data.groupby(['customer_id','age']).agg( tran_count = ('t_dat','nunique'),
                                       article_count = ('article_id','nunique'),
                                       spend = ('price','sum')
                                      ).reset_index()

In [101]:
cust_features_1 = pd.pivot_table(cust_data[['customer_id','product_group_name','t_dat']],
         index = 'customer_id',
         columns = 'product_group_name',
         values = 't_dat',
         aggfunc = 'nunique'
        ).fillna(0).reset_index()

In [102]:
cust_features = pd.merge(cust_features, 
                         cust_features_1,
                         left_on = 'customer_id',
                         right_on = 'customer_id',
                        )

In [103]:
cust_features.head()

,customer_id,age,tran_count,article_count,spend,Accessories,Bags,Cosmetic,Fun,Furniture,...,Interior textile,Items,Nightwear,Shoes,Socks & Tights,Stationery,Swimwear,Underwear,Underwear/nightwear,Unknown
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,49.0,10,19,0.648983,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,24.0,7,14,0.704780,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0
2,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,20.0,3,6,0.166000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,000172a9c322560c849754ffbdfdb2180d408aa7176b94...,45.0,7,23,0.565203,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,44.0,144,555,25.466898,23.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,14.0,7.0,0.0,10.0,14.0,0.0,1.0


In [105]:
cust_data_2020_9.iloc[:2]['prods'].apply(lambda x : list(x)).values

array([list(['a176209023', 'a568601006', 'a568601043', 'a607642008', 'a625548001', 'a627759010', 'a656719005', 'a694736004', 'a697138006', 'a745232001', 'a785186005', 'a785710001']),
       list(['a351484002', 'a541518023', 'a578020002', 'a663713001', 'a723529001', 'a727808001', 'a727808007', 'a750424014', 'a794321007', 'a851400006', 'a852643001', 'a852643003'])],
      dtype=object)

In [106]:
cust_data_2020_9["prods"] = cust_data_2020_9["prods"].apply( lambda x : list(x) ) 
cust_data_2020_9["prods"].values[:5]

array([list(['a176209023', 'a568601006', 'a568601043', 'a607642008', 'a625548001', 'a627759010', 'a656719005', 'a694736004', 'a697138006', 'a745232001', 'a785186005', 'a785710001']),
       list(['a351484002', 'a541518023', 'a578020002', 'a663713001', 'a723529001', 'a727808001', 'a727808007', 'a750424014', 'a794321007', 'a851400006', 'a852643001', 'a852643003']),
       list(['a448509014', 'a715624008', 'a719530003', 'a726925001', 'a735843004', 'a783388001']),
       list(['a181448103', 'a309864002', 'a309864013', 'a380609037', 'a471714036', 'a519583033', 'a554811019', 'a554811021', 'a556528009', 'a562245064', 'a619234011', 'a661794002']),
       list(['a297791001', 'a304692003', 'a536933002', 'a668537001', 'a676364002', 'a734943003', 'a772659001', 'a779924003', 'a817352006', 'a853816001', 'a865520001', 'a886540001'])],
      dtype=object)

In [107]:
model_data = pd.merge(cust_data_2020_9,
                      cust_features,
                      left_on = 'customer_id',
                      right_on = 'customer_id'
                     )

In [108]:
test_split = 0.1

# Initial train and test split.
train_df, test_df = train_test_split(
    model_data,
    test_size=test_split,
)

# Splitting the test set further into validation
# and new test sets.
val_df = test_df.sample(frac=0.5)
test_df.drop(val_df.index, inplace=True)

print(f"Number of rows in training set: {len(train_df)}")
print(f"Number of rows in validation set: {len(val_df)}")
print(f"Number of rows in test set: {len(test_df)}")

Number of rows in training set: 169767
Number of rows in validation set: 9432
Number of rows in test set: 9431


In [109]:
prods = tf.ragged.constant(train_df["prods"].values)
lookup = tf.keras.layers.StringLookup(output_mode="multi_hot")
lookup.adapt(prods)
vocab = lookup.get_vocabulary()


def invert_multi_hot(encoded_labels):
    """Reverse a single multi-hot encoded label to a tuple of vocab terms."""
    hot_indices = np.argwhere(encoded_labels == 1.0)[..., 0]
    return np.take(vocab, hot_indices)


print("Vocabulary:\n")
print(vocab)

Vocabulary:

['[UNK]', 'a372860001', 'a372860002', 'a448509014', 'a464297007', 'a706016001', 'a399223001', 'a399256001', 'a610776002', 'a351484002', 'a156231001', 'a562245046', 'a158340001', 'a160442007', 'a568601006', 'a706016002', 'a484398001', 'a610776001', 'a448509001', 'a579541001', 'a160442010', 'a399256005', 'a507909001', 'a562245001', 'a111586001', 'a673677002', 'a111593001', 'a537116001', 'a608776002', 'a554450001', 'a179123001', 'a179950001', 'a536139006', 'a673396002', 'a228257001', 'a253448003', 'a573716012', 'a554598001', 'a399223029', 'a297067002', 'a562245018', 'a189616006', 'a436261001', 'a751471001', 'a507910001', 'a200182001', 'a706016003', 'a573085028', 'a590928001', 'a568597006', 'a556539003', 'a516859008', 'a372860024', 'a108775015', 'a305931001', 'a759871002', 'a572797001', 'a408875001', 'a456163060', 'a547780001', 'a278811006', 'a685814001', 'a470789001', 'a448509018', 'a556539001', 'a678942001', 'a599580017', 'a399223033', 'a636323001', 'a611415001', 'a253448001

In [110]:
def make_model():
    shallow_mlp_model = keras.Sequential(
        [
            layers.Dense(512, activation="relu"),
            layers.Dense(256, activation="relu"),
            layers.Dense(lookup.vocabulary_size(), activation="sigmoid"),
        ]  # More on why "sigmoid" has been used here in a moment.
    )
    return shallow_mlp_model

In [ ]:
epochs = 20

shallow_mlp_model = make_model()
shallow_mlp_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["categorical_accuracy"]
)

history = shallow_mlp_model.fit(
    train_dataset, validation_data=validation_dataset, epochs=epochs
)

In [ ]:
top_12_products = transactions_train[(transactions_train['t_year']==2020)&
                                     (transactions_train['t_month'].isin([9]))
                                    ].groupby('article_id').agg({'customer_id':'nunique'}).reset_index().\
sort_values('customer_id', ascending=False).head(12)

top_12_products['article_id'] = " 0"+top_12_products.article_id.astype('str')

In [ ]:
submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

submission['prediction'] = top_12_products.article_id.sum()

submission.to_csv('submission.csv', index = False)